In [1]:
# Pandas Task 1
URL = 'hw_2_user_data.csv'
COLS = ['UserID','UserName','WatchedMovie','MovieGenre','SessionLength','LastLoginDate']

import pandas as pd

pa_df = pd.read_csv(URL, parse_dates=['LastLoginDate'])

# The fields 'UserName' and 'MovieGenre' were parsed as type 'object' by pandas.
# In order to use the properly, we will convert these types to python string.
pa_df['UserName'] = pa_df['UserName'].astype('string')
pa_df['MovieGenre'] = pa_df['MovieGenre'].astype('string')
# pa_df


print(f'DATA TYPES: \n------------\n{pa_df.dtypes}')
print(f'\nDATA: \n------------\n')
pa_df

DATA TYPES: 
------------
Unnamed: 0                int64
UserID                    int64
UserName         string[python]
WatchedMovie               bool
MovieGenre       string[python]
SessionLength             int64
LastLoginDate    datetime64[ns]
dtype: object

DATA: 
------------



,Unnamed: 0,UserID,UserName,WatchedMovie,MovieGenre,SessionLength,LastLoginDate
0,0,34557,Leslie Shelton,False,<NA>,175,2012-04-30
1,1,48013,Hannah Sanders,False,<NA>,1409,2020-05-30
2,2,13230,Christopher Torres,True,Adventure Drama,181,2009-11-25
3,3,18988,Christopher Stokes,True,Animation Adventure Comedy Family Fantasy Musi...,179,2022-02-15
4,4,29844,Joel Cox,False,<NA>,227,2012-06-09
...,...,...,...,...,...,...,...
199995,199995,24547,Mark Rivera,True,Drama Romance Sci-Fi,1280,2022-03-20
199996,199996,12542,Jeremy Gregory,False,<NA>,1433,2008-10-01
199997,199997,12153,Krista Bush,False,<NA>,390,2008-05-14
199998,199998,29021,Suzanne Johnson,True,Biography Drama,160,2017-03-12


In [2]:
# PySpark Task 1
from pyspark.sql.functions import to_date, months_between, current_date, datediff, col
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, BooleanType

spark = SparkSession.builder.getOrCreate()

# Ignore all logs except errors when running this script on pyspark (i.e. spark-submit)
spark.sparkContext.setLogLevel('ERROR')

# Task 1
# Read json dataset into a DataFrame and apply the schema defined above.
df = spark.read.csv('hw_2_user_data.csv', header=True) 

# Rename columns. 
df = df.withColumnRenamed('_c0', 'Index')\
       .withColumn('UserID', df['UserID'].cast(IntegerType()))\
       .withColumn('SessionLength', df['SessionLength'].cast(IntegerType()))\
       .withColumn('WatchedMovie', df['WatchedMovie'].cast(BooleanType()))\
       .withColumn('LastLoginDate', to_date(df['LastLoginDate']))



# Display the updated schema 
df.printSchema()
# Display data
df.show()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/07 22:44:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- Index: string (nullable = true)
 |-- UserID: integer (nullable = true)
 |-- UserName: string (nullable = true)
 |-- WatchedMovie: boolean (nullable = true)
 |-- MovieGenre: string (nullable = true)
 |-- SessionLength: integer (nullable = true)
 |-- LastLoginDate: date (nullable = true)

+-----+------+------------------+------------+--------------------+-------------+-------------+
|Index|UserID|          UserName|WatchedMovie|          MovieGenre|SessionLength|LastLoginDate|
+-----+------+------------------+------------+--------------------+-------------+-------------+
|    0| 34557|    Leslie Shelton|       false|                NULL|          175|   2012-04-30|
|    1| 48013|    Hannah Sanders|       false|                NULL|         1409|   2020-05-30|
|    2| 13230|Christopher Torres|        true|     Adventure Drama|          181|   2009-11-25|
|    3| 18988|Christopher Stokes|        true|Animation Adventu...|          179|   2022-02-15|
|    4| 29844|          Joel C

In [3]:
# Pandas Task 2
# movie_genres = df['MovieGenre'].unique().dropna()

# First we find rows where a movie was watched, 
# then group this data by genre 
# and then get the count for each genre.
pa_df[pa_df['WatchedMovie'] == True]\
    .groupby(['MovieGenre'])\
    .agg({'WatchedMovie': ['count']})



,WatchedMovie
,count
MovieGenre,
Action Adventure,1557
Action Adventure Comedy Crime Thriller,1624
Action Adventure Crime Drama,1577
Action Adventure Crime Mystery Thriller,1643
Action Adventure Drama,1583
...,...
Drama Mystery Thriller,1616
Drama Romance,1608


In [4]:
# PySpark Task 2
# Calculate the total number of watched movies for each genre.
df.filter(df.WatchedMovie == True).groupBy('MovieGenre').count()\
       .orderBy('MovieGenre').show()

+--------------------+-----+
|          MovieGenre|count|
+--------------------+-----+
|    Action Adventure| 1557|
|Action Adventure ...| 1624|
|Action Adventure ...| 1577|
|Action Adventure ...| 1643|
|Action Adventure ...| 1583|
|Action Adventure ...| 1598|
|Action Adventure ...| 1603|
|Action Adventure ...| 1582|
|Action Adventure ...| 1553|
|Action Adventure ...| 1598|
|Action Adventure ...| 1628|
|Action Adventure ...| 1583|
|Action Biography ...| 1642|
|        Action Crime| 1565|
|Action Crime Dram...| 1580|
|Action Crime Fant...| 1506|
|Action Crime Thri...| 1563|
|Action Drama Thri...| 1553|
|    Action Drama War| 1600|
|Action Mystery Sc...| 1609|
+--------------------+-----+
only showing top 20 rows



In [5]:
# Pandas Task 3
import numpy as np

# df['nb_months'] = ((df.date2 - df.date1)/np.timedelta64(1, 'M'))
# Find records where today - LastLoginDate is ~3 months. 
# For this we will find the time delta and devide by 30 day period to get the difference in months.
# Then we can filter rows out where difference is larger than 3.

len(pa_df[((pd.to_datetime('today') - pa_df['LastLoginDate']) / np.timedelta64(1, 'D')) < 92]\
    ['UserID'].unique())

# Note that we're using unit 'D' instead of unit 'M' as 'M' is no longer supported by numpy. 
# And we're using # of days so a 3 month period comes around to ~92

1878

In [6]:
# PySpark Task 3
# Identify the number of unique users who logged in during the last three months.
df.filter(months_between(current_date(), df.LastLoginDate) <= 3)\
       .select('UserID').distinct().count()

1878

In [7]:
# Pandas Task 4

# Filter by watched movies and group by UserID while aggregating count for 
# watched movies and avg session length. Then filter results where watched count > 2.

r = pa_df[pa_df['WatchedMovie'] == True].groupby(['UserID'])\
    .agg({'WatchedMovie': ['count'], 'SessionLength': ['mean']})
r[r['WatchedMovie']['count'] > 2]

,WatchedMovie,SessionLength
,count,mean
UserID,,
2,3,839.000000
6,4,783.750000
9,3,788.666667
13,4,998.500000
17,3,979.666667
...,...,...
49977,3,497.666667
49983,6,453.833333


In [8]:
# PySpark Task 4
# Determine the average session length for users who have watched more than two movies.

# find users who have watched 
df.filter(df.WatchedMovie == True)\
    .groupBy('UserID').agg({'WatchedMovie': 'count', 'SessionLength': 'mean'})\
    .where(col('count(WatchedMovie)') > 2).orderBy('UserID').show()


+------+-------------------+------------------+
|UserID|count(WatchedMovie)|avg(SessionLength)|
+------+-------------------+------------------+
|     2|                  3|             839.0|
|     6|                  4|            783.75|
|     9|                  3| 788.6666666666666|
|    13|                  4|             998.5|
|    17|                  3| 979.6666666666666|
|    20|                  4|             737.0|
|    21|                  3|             702.0|
|    22|                  5|             490.0|
|    26|                  3| 350.6666666666667|
|    31|                  5|             447.8|
|    32|                  3| 695.3333333333334|
|    35|                  3| 840.6666666666666|
|    36|                  4|            454.75|
|    37|                  3| 762.6666666666666|
|    38|                  4|            631.75|
|    40|                  4|             717.5|
|    41|                  3| 484.6666666666667|
|    44|                  3| 419.6666666

In [9]:
# Pandas Task 5
# Insert new column by computing time delta (#days) between today and LastLoginDate.
pa_df.insert(7, 'DaysSinceLastLogin', ((pd.to_datetime('today') - pa_df['LastLoginDate']) / np.timedelta64(1, 'D')))

# Cast DaysSinceLastLogin to int for better readability.
pa_df['DaysSinceLastLogin'] = pa_df['DaysSinceLastLogin'].astype(int)

pa_df

,Unnamed: 0,UserID,UserName,WatchedMovie,MovieGenre,SessionLength,LastLoginDate,DaysSinceLastLogin
0,0,34557,Leslie Shelton,False,<NA>,175,2012-04-30,4360
1,1,48013,Hannah Sanders,False,<NA>,1409,2020-05-30,1408
2,2,13230,Christopher Torres,True,Adventure Drama,181,2009-11-25,5247
3,3,18988,Christopher Stokes,True,Animation Adventure Comedy Family Fantasy Musi...,179,2022-02-15,782
4,4,29844,Joel Cox,False,<NA>,227,2012-06-09,4320
...,...,...,...,...,...,...,...,...
199995,199995,24547,Mark Rivera,True,Drama Romance Sci-Fi,1280,2022-03-20,749
199996,199996,12542,Jeremy Gregory,False,<NA>,1433,2008-10-01,5667
199997,199997,12153,Krista Bush,False,<NA>,390,2008-05-14,5807
199998,199998,29021,Suzanne Johnson,True,Biography Drama,160,2017-03-12,2583


In [10]:
# PySpark Task 5
# Add a new column indicating the days since the last login for each user.

df = df.withColumn('DaysSinceLastLogin', datediff(current_date(), df.LastLoginDate)) 
# df.printSchema()
df.show()

+-----+------+------------------+------------+--------------------+-------------+-------------+------------------+
|Index|UserID|          UserName|WatchedMovie|          MovieGenre|SessionLength|LastLoginDate|DaysSinceLastLogin|
+-----+------+------------------+------------+--------------------+-------------+-------------+------------------+
|    0| 34557|    Leslie Shelton|       false|                NULL|          175|   2012-04-30|              4360|
|    1| 48013|    Hannah Sanders|       false|                NULL|         1409|   2020-05-30|              1408|
|    2| 13230|Christopher Torres|        true|     Adventure Drama|          181|   2009-11-25|              5247|
|    3| 18988|Christopher Stokes|        true|Animation Adventu...|          179|   2022-02-15|               782|
|    4| 29844|          Joel Cox|       false|                NULL|          227|   2012-06-09|              4320|
|    5| 12305|       Jerry Perez|       false|                NULL|          288